<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Guided Problem Set 16: Ideal Gas Law from Molecular Simulation</h1>


<a name='section_16_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P16.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_16_1">P16.1 Gay-Lussac's Law Part I</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_16_1">P16.1 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_16_2">P16.2 Gay-Lussac's Law Part II</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_16_2">P16.2 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_16_3">P16.3 Avogadro's Law and Boyle's Law</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_16_3">P16.3 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_16_4">P16.4 Putting it all Together</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_16_4">P16.4 Problems</a></td>
    </tr>
</table>



<h3>Overview</h3>

In this problem set, we will use the simulation completed in the previous Pset to derive the properties of an ideal gas.

The ideal gas law combines Gay-Lussac's, Avogadro's, and Boyle's Laws. We will confirm each of them by using the output of our molecular simulation to compute macroscopic quantities such as pressure based on the microscopic components (i.e. the individual particles).

In real life experiments, it's often easier to keep the pressure constant at normal atmospheric pressure and vary the volume of a system. In simulations, however, it's easier to keep the volume constant and measure the change in pressure when varying other properties of the system such as the number of particles and the temperature.

In the case of Avogadro's law, for example, we will show a version where we keep the volume constant and vary the number of particles, while observing the pressure.   

<h3>Importing Libraries</h3>

Before beginning, run the code cells below to import the relevant libraries for this notebook.

In [ ]:
#>>>RUN: P16.0-runcell00

#install the following:

#!pip install imageio
#!pip3 install torch torchvision torchaudio
#!pip install lmfit

In [ ]:
#>>>RUN: P16.0-runcell01

import itertools
from IPython.display import HTML
from scipy.integrate import odeint
import matplotlib.patches as patches
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
import imageio
from IPython.display import Image
import random
from lmfit import Model, Parameter, report_fit, fit_report
from lmfit.models import LinearModel

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.

In [ ]:
#>>>RUN: P16.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_16_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P16.1 P16.1 Gay-Lussac's Law Part I</h2>    

| [Top](#section_16_0) | [Previous Section](#section_16_0) | [Problems](#problems_16_1) | [Next Section](#section_16_2) |

<h3>Note: Redefine Functions</h3>

We will be using functions here that you will have defined in Pset 15. In the cell below, again define the following functions (see solutions to P16 for

In [ ]:
#>>>RUN: P16.1-runcell00

#FROM PROBLEM: P15.1.1
def wall_collision(pos, vel):

    #collision with the right wall
    vel[pos[:, 0] > 1, 0] = #Your code goes here

    #collision with the left wall
    vel[pos[:, 0] < 0, 0] = #Your code goes here

    #collision with the top wall
    vel[pos[:, 1] > 1, 1] = #Your code goes here

    #collision with the bottom wall
    vel[pos[:, 1] < 0, 1] = #Your code goes here
    return vel

#FROM PROBLEM: P15.2.1
def compute_distance_pairs(position, idx_pair):
    delta_x = #Your code goes here
    delta_y = #Your code goes here
    distances = #np.sqrt(delta_x**2 + delta_y**2)
    return distances

#FROM PROBLEM: P15.2.2
def which_pair_collides(distance_pairs):
    #Your code goes here
    pass

#FROM PROBLEM: P15.2.3
def pair_collision(pos_1i, pos_2i, vel_1i, vel_2i):
    vel_1f = 0 #YOUR CODE HERE
    vel_2f = 0 #YOUR CODE HERE
    return vel_1f, vel_2f


<h3>Overview</h3>

First, we will check Gay-Lussac's law, which states that the pressure exerted by a given mass and constant volume of an ideal gas on the sides of its container is directly proportional to its absolute temperature. We'll run the simulation as we've done before, using a fixed-size box filled with a fixed number of particles, and use the output to determine whether or not $P \propto T$. We will then fit the data with a linear model to obtain the slope, $P/T = A$ where $A$ is a constant.

We already know how to find the temperature based on the average of the squared velocities of the individual particles in the simulation.

$$d \times \frac{1}{2}k_B T = 2 \times \frac{1}{2}k_B T = k_B T = KE_{avg} = \frac{1}{2}m\left<{v^2}\right> $$

$$T = \frac{m \left<{v^2}\right>}{2 k_B}$$

Given conservation of energy, the average of the squared velocity $\left<{v^2}\right>$ can be easily calculated using the initial constant velocity magnitude that we set for every particle.

Now, we need to find a way to determine the pressure $P$, which is defined as:

$$P = \frac{F}{4L}$$

where $F$ is the force and $L$ is the side length of the box. The factor of 4 comes from the fact that we will not distinguish between the 4 walls of the box, but rather will sum the total force exerted on all of them. Note that this is the pressure as defined in 2D space, as opposed to the more familiar force per unit area used in normal 3D space.

We just need to compute the force imparted by the particles hitting the sides. Noting that $\vec{F} = \Delta \vec{p} / \Delta t$, where $\vec{p}$ is the change in momentum, we can compute the force after reaching thermal equilibrium as follows:

 * Track the particles that hit the walls of the box at each time step and compute their change in momentum $\Delta p_i = m \Delta v_i$.
 * This change in momentum is due to the force the walls exert on the particles, which must be equal and opposite to the force the particles exert on the walls.
 * The force is then $F = \sum_i |\Delta p_i| / \Delta t$ where $\Delta t$ is the length of time over which we sum the momentum change,

Therefore,

 $$P = \frac{m \sum |\Delta v_i|}{4L \Delta t}$$

Both $P$ and $T$ depend linearly on the particle mass $m$. Since this term cancels for $P/T$, we will, in practice, calculate $P/m$ and $T/m$.

<a name='problems_16_1'></a>     

| [Top](#section_16_0) | [Restart Section](#section_16_1) | [Next Section](#section_16_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 16.1.1</span>

Write a function `compute_delta_v` that takes 2 inputs: `vel_i`, the velocity before a collision with the wall and `vel_f`, the velocity after the collision, and returns the sum of the change of velocity magnitude of all the collisions with the wall:

$$\sum_i |\Delta v_i|$$



In [ ]:
#>>>PROBLEM: P16.1.1

def compute_delta_v(vel_i, vel_f):
    result = #YOUR CODE HERE
    return result

<h3>Augmenting the Simulation</h3>

Let's add this function to our simulation code to keep track of $\Delta v$

In [ ]:
#>>>RUN: P16.1-runcell01

def simulate(pos, vel, time_steps, dt):


    nparticles = pos.shape[0]
    indices = np.arange(0,n_particles,1)
    idx_pair = np.array(list(itertools.combinations(indices,2)))

    pos_simulated = np.zeros((time_steps, n_particles, 2))
    vel_simulated = np.zeros((time_steps, n_particles, 2))
    dv = np.zeros((time_steps))

    pos_simulated[0] = pos
    vel_simulated[0] = vel
    dv[0] = 0

    for i in range(1,time_steps):

        collision_idx = which_pair_collides(compute_distance_pairs(pos, idx_pair))
        v1f, v2f = pair_collision(pos[idx_pair[collision_idx][:,0]], pos[idx_pair[collision_idx][:,1]],
                                  vel[idx_pair[collision_idx][:,0]], vel[idx_pair[collision_idx][:,1]])

        vel[idx_pair[collision_idx][:,0]] = v1f
        vel[idx_pair[collision_idx][:,1]] = v2f

        vel_before = vel.copy()
        vel = wall_collision(pos, vel)
        delta_v = compute_delta_v(vel_before, vel)
        dv[i] = delta_v

        pos = pos + vel * dt


        vel_simulated[i] = vel
        pos_simulated[i] = pos

    return pos_simulated, vel_simulated, dv



### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 16.1.2</span>

Now, write a function named `compute_pressure` that takes 3 inputs: an array `dv_sim` which keeps track of the $\Delta v$ in each simulation step, `npoints` the number of time steps we want to integrate over working backwards from the last step in the simulation, and `dt` the length of the time step, and returns the pressure using the formula given in the overview section:

$$P/m = \frac{ \sum |\Delta v_i|}{4L \Delta t}$$

Note that we are using $L=1$ in our simulation.

In [ ]:
#>>>PROBLEM: P16.1.2

def compute_pressure(dv_sim, npoints, dt):
    result = #YOUR CODE HERE
    return result


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 16.1.3</span>

To check that your code is working properly, the following code cell uses the function `compute_pressure` to find pressures for initial velocities of $0$, $300$, $350$, $400$, and $450~m/s$. For your answer, enter the pressure for the case when `v_magnitude` is equal to $300 m/s$. Use precision `7e5`. This will take a while!

In [ ]:
#>>>PROBLEM: P16.1.3

np.random.seed(673720454)
n_particles = 400 #number of particles
pos_initial = np.random.rand(n_particles, 2)
particle_radius = 0.0002

npoints = 2000
simulation_steps = 40000
dt = 0.0000008

pressure_list = []
v_list = np.array([0, 300, 350, 400, 450])
for v_magnitude in v_list:
    np.random.seed(673720454)
    vel_initial = np.zeros((n_particles,2))
    thetas = np.random.rand(n_particles)* 2 * np.pi
    vel_initial[:, 0] = np.cos(thetas) * v_magnitude
    vel_initial[:, 1] = np.sin(thetas) * v_magnitude
    pos_sim, vel_sim, dv_sim = simulate(pos_initial, vel_initial, simulation_steps, dt)
    pressure = compute_pressure(dv_sim, npoints, dt)
    pressure_list.append(pressure)

print(pressure_list)

<a name='section_16_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P16.2 Gay-Lussac's Law Part II</h2>    

| [Top](#section_16_0) | [Previous Section](#section_16_1) | [Problems](#problems_16_2) | [Next Section](#section_16_3) |

<a name='problems_16_2'></a>     

| [Top](#section_16_0) | [Restart Section](#section_16_2) | [Next Section](#section_16_3) |

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 16.2.1</span>

Write a function named `compute_temperature` that takes 1 input: an array `v_list` of magnitudes of velocities, and returns an array of temperatures using the formula given in the overview section. Use `k_B=1.38e-23` J/K.

$$T/m = \frac{\left<{v^2}\right>}{2 k_B}$$

With those numbers, we can plot P vs. T to see if it looks linear. Try it out!

In [ ]:
#>>>PROBLEM: P16.2.1

def compute_temperature(vel):
    result = #YOUR CODE HERE
    return result

#MAKE A PLOT
T = compute_temperature(v_list)
plt.plot(T, pressure_list)
plt.xlabel('Temperature')
plt.ylabel('Pressure')
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 16.2.2</span>

Given the values for pressure versus temperature from the simulation (which look very close to linearly correlated), use `lmfit` to get the slope of the $P$ vs $T$ graph. In order to make the numbers less humongous, and thereby avoid possible numerical problems in the fit, scale the temperatures and pressures by $10^{-27}$ and $10^{-7}$, respectively. Fit the data with a linear model, allowing for both a slope and intercept. Enter the fitted slope with precision `0.05`.

In [ ]:
#>>>PROBLEM: P16.2.2

import lmfit
from lmfit import Model

#YOUR CODE HERE

<a name='section_16_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P16.3 Avogadro's Law and Boyle's Law</h2>    

| [Top](#section_16_0) | [Previous Section](#section_16_2) | [Problems](#problems_16_3) | [Next Section](#section_16_4) |

<a name='problems_16_3'></a>     

| [Top](#section_16_0) | [Restart Section](#section_16_3) | [Next Section](#section_16_4) |

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 16.3.1</span>

The next step is to confirm the pressure version of Avogadro's Law, which states that $P \propto N$ (where $N$ is the number of particles) when the temperature and volume are held constant. The previous simulation had a fixed number of particles and varied their initial velocities (and hence the temperature). Now, we need to rerun the entire simulation, but this time varying the number of particles while keeping their initial velocities fixed.

Edit the code to complete the simulation and run it for the 5 values of $N$. For your answer, enter the value of pressure when the number of particles is equal to 450. Use precision `1e7`. This will take a while!


In [ ]:
#>>>PROBLEM: P16.3.1

np.random.seed(673720454)
n_particles = 400 #number of particles
pos_initial = np.random.rand(n_particles, 2)
particle_radius = 0.0002

npoints = 2000
simulation_steps = 40000
dt = 0.0000008
nparticles_list = [300,350,400,450,500]
pressure_list = []
v_magnitude= 500

for n_particles in nparticles_list:
    np.random.seed(673720454)
    #YOUR CODE HERE
    pressure_list.append(pressure)

print(pressure_list[3])

#make a plot!
plt.plot(nparticles_list, pressure_list)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 16.3.2</span>

The final step is to check Boyle's Law which states that $P \propto {1}/{V}$ for fixed  number of particles and temperature. Note that in a 2D simulation, we'll actually change the area, not the volume, by varying the side length variable `L`.

Edit the code for the function `wall_collision` so that you can vary the wall side length `L`. As before, assume that both `pos` and `vel` have shape `[n_particles, 2]`, but now allow the side length `L` to be a global variable, defined outside of this function.


In [ ]:
#>>>PROBLEM: P16.3.2

def wall_collision(pos, vel):
    #collision with the right wall
    vel[pos[:, 0] > L, 0] = # Your code goes here.

    #collision with the left wall
    vel[pos[:, 0] < 0, 0] = # Your code goes here.

    #collision with the top wall
    vel[pos[:, 1] > L, 1] = # Your code goes here.

    #collision with the bottom wall
    vel[pos[:, 1] < 0, 1] = # Your code goes here.
    return vel

#-----------------------------------------------------
# CHECK YOUR RESULT

L = 2.

# Sample positions (x, y) of particles
pos_initial = np.array([
    [0.5, 0.5],  # Inside the box
    [L+0.5, 0.5],  # Colliding with the right wall
    [L-0.5, 0.5], # Colliding with the left wall
    [0.5, L+0.5],  # Colliding with the top wall
    [0.5, L-0.5]  # Colliding with the bottom wall
])

# Sample velocities (vx, vy) of particles
vel_initial = np.array([
    [0.1, 0.1],  # Random velocity
    [0.2, 0.2],  # Random velocity
    [-0.3, -0.3],# Random velocity
    [0.4, 0.4],  # Random velocity
    [-0.5, -0.5] # Random velocity
])

print("Original Velocities:\n", vel_initial)

# Apply the wall_collision function to the sample data
updated_vel = wall_collision(pos_initial, vel_initial)

# Print the updated velocities after collision handling
print("\nUpdated Velocities after Collision Handling:\n", updated_vel)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 16.3.3</span>

Run the simulation below and enter the pressure value when the container has side length $L=2$. Use precision `1e-3`. This will take a while!
    
Then try plotting $P$ vs $\frac{1}{V}$.


In [ ]:
#>>>PROBLEM: P16.3.3

np.random.seed(673720454)
n_particles = 400 #number of particles
pos_initial = np.random.rand(n_particles, 2)
particle_radius = 0.0002

npoints = 2000
simulation_steps = 40000
dt = 0.0000008
pressure_list = []
L_list = np.array([1,1.5,2,2.5,3])

for L in L_list:
    vel_initial = np.zeros((n_particles,2))
    np.random.seed(673720454)
    thetas = np.random.rand(n_particles)* 2 * np.pi
    vel_initial[:, 0] = np.cos(thetas) * v_magnitude
    vel_initial[:, 1] = np.sin(thetas) * v_magnitude
    pos_sim, vel_sim, dv_sim = simulate(pos_initial, vel_initial, simulation_steps, dt)
    pressure = compute_pressure(dv_sim, npoints, dt)
    pressure_list.append(pressure)
    #print(pressure)
    
print(pressure_list[2])

#Make a plot!
plt.plot(1/(L_list**2), pressure_list)

<a name='section_16_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P16.4 Putting it all Together</h2>    

| [Top](#section_16_0) | [Previous Section](#section_16_3) | [Problems](#problems_16_4) |

<a name='problems_16_4'></a>     

| [Top](#section_16_0) | [Restart Section](#section_16_4) |

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 16.4.1</span>

We've confirmed that:
* with $N$ and $V$ fixed, $P \propto T$
* with $V$ and $T$ fixed, $P \propto N$
* with $T$ and $N$ fixed, $P \propto {1}/{V}$

Thus, combining them all, we get that:

$$\dfrac{PV}{NT} = C,$$

where $C$ is a constant.

Rearranging this equation gives:

$$\dfrac{P}{T} = C\dfrac{N}{V}$$

In Problem 16.2.2, by fitting we got the value of $\frac{P}{T}$ with 400 particles and box of side length $1$. Using this fit value, compute the value of $C$. Report your answer as a number in units of J/K `1e-23`, with precision `1e-1`.

Hint. With 400 particles and the volume(area) = $1\times 1 = 1$, the slope in Problem 16.2.2 is $400C$

The ideal gas law:

$$PV = Nk_B T$$

is just a rearrangement of what we listed above, which means that the value we found for $C$ should equal the Boltzmann Constant $k_{B}$. How close did our seemingly very simple simulation get to predicting this fundamental property of gasses?

